In [1]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, stream, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization as BatchNorm
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [2]:
def train_network():
    """ Train a Neural Network to generate music"""
    notes = get_notes()
    
    print(notes)
    # get amount of pitch names
    n_vocab = len(set(notes))
    
    print(n_vocab)
    
    network_input, network_output = prepare_sequences(notes, n_vocab)
    
    print(network_input)
    print(network_output)
    
    model = create_network(network_input, n_vocab)
    
    model.summary()
    
    train(model, network_input, network_output)

In [3]:
def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []
    
    for file in glob.glob("midi_songs/*.mid"):
        midi = converter.parse(file)
        
        print("Parsing %s" % file)
        
        notes_to_parse = None
        
        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes
            
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
    
    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)
        
    return notes

In [4]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 32
    
    # get all pitch names
    pitchnames = sorted(set(item for item in notes))
    
    # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    
    network_input = []
    network_output = []
    
    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])
        
    n_patterns = len(network_input)
    
    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # normalize input
    network_input = network_input / float(n_vocab)
    
    network_output = np_utils.to_categorical(network_output)
    
    return (network_input, network_output)

In [5]:
def create_network(network_input, n_vocab):
    """ Create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    # model.load_weights('lofi-hip-hop-weights-improvement-100-0.6290.hdf5')
    
    return model

In [6]:
def train(model, network_input, network_output):
    """ Train the neural network """
    filepath = "lofi-thai-weights-improvement-1{epoch:02d}-{loss:.4f}.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]
    
    model.fit(network_input, network_output, epochs=10, batch_size=48, callbacks=callbacks_list)

In [7]:
train_network()

Parsing midi_songs/LofiPianoSample1.mid
['11.0.4', '4.7.9', 'D6', '11.2.6', '0.2.6', 'A5', 'G5', 'F#5', '7.9.11.2', 'D5', 'E5', 'A5', 'F#5', 'E5', 'D5', 'C#5', '9.11.2', 'B4', 'D5', 'E5', '6.7.11', '2.4', 'G5', 'A5', '11.1.6', '7.9', 'E5', 'D5', '1.6', '1.2.4', 'A4', 'B4', 'E5', 'C#5', 'B5', '0.4', '7.9.11', 'D6', '11.2.6', '0.2.6', 'A5', 'G5', 'F#5', '7.9.11.2', 'D5', 'E5', 'A5', 'F#5', 'E5', 'D5', 'C#5', '9.11.2', 'B4', 'D5', 'E5', '6.7.11', '2.4', 'G5', 'A5', '11.1.6', '7.9', 'D5', 'E5', 'F#5', '1.2', 'B4', '6.9', 'C#5', 'B5', '4.6', 'C#6', '10.2', 'A5', '4.6', '11.2.6', 'D6', 'C#6', 'B5', '0.4', '7.9.11', 'A5', 'G5', '6.10.1', '6.7', 'E5', 'D5', 'D6', '1.4.7', 'C#6', 'D6', 'F#6', 'D6', 'C#5', 'G5', 'C#6', 'G5', 'F#5', 'D6', 'C#6', 'A3', 'C#5', 'D5', 'G4', 'E6', 'F#4', 'A3', 'A2', 'F#6', '1.2', 'C#3', '11.1', 'B5', 'F#3', 'A5', 'D3', 'B-3', 'C#5', 'F#4', 'B-4', 'E4', 'D4', 'B-5', 'C#4']
48
[[[0.14583333]
  [0.27083333]
  [0.79166667]
  ...
  [0.0625    ]
  [0.5       ]
  [0.60416667